In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND,  either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK for Python: AutoML训练视频物体跟踪模型用于批量预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_video_object_tracking_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_video_object_tracking_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/sdk_automl_video_object_tracking_batch.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示如何使用Python的Vertex AI SDK创建视频对象跟踪模型，并使用Google Cloud的AutoML模型进行批量预测。

了解有关视频数据对象跟踪的更多信息。

### 目标

在本教程中，您将学习如何从Python脚本创建一个AutoML视频对象跟踪模型，然后使用Vertex AI SDK进行批量预测。您也可以选择使用`gcloud`命令行工具或在云控制台上在线创建和部署模型。

本教程使用以下Google Cloud ML服务和资源：

- Vertex AI

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

使用批量预测和在线预测之间有一个关键区别：

* 预测服务：针对整个实例集（即一个或多个数据项）进行即时预测并实时返回结果。

* 批量预测服务：对整个实例集进行后台排队（批量）预测，并在准备就绪时将结果存储在Cloud Storage桶中。

数据集

本教程使用的数据集是Traffic数据集。您在本教程中使用的数据集版本存储在公共云存储桶中。

费用

本教程使用谷歌云的可计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储 价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成费用估算。

## 安装

安装以下所需的软件包以执行此笔记本。

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage

### 仅限Colab：取消下面的单元格注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

开始之前

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下步骤：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的谷歌云账号

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关指示进行操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经进行了身份验证。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 请查看如何为您的服务账户授予云存储权限，请访问https://cloud.google.com/storage/docs/gsutil/commands/iam＃ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在时才运行以下代码以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 导入库并定义常数

In [ ]:
import json
import os

import google.cloud.aiplatform as aiplatform
from google.cloud import storage

初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# 教程

现在您已经准备好开始创建自己的AutoML视频对象跟踪模型。

#### 云存储训练数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/ai-platform-unified/video/traffic/traffic_videos_labels.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的Traffic数据集的一种版本，使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

! gsutil cat $FILE | head

### 创建数据集

接下来，使用`VideoDataset`类的`create`方法创建`Dataset`资源，需要以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件列表，用于将数据项导入`Dataset`资源中。

此操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.VideoDataset.create(
    display_name="Traffic",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.object_tracking,
)

print(dataset.resource_name)

### 创建和运行训练管道

要训练AutoML模型，您需要执行两个步骤：1）创建训练管道，2）运行管道。

#### 创建训练管道

使用`AutoMLVideoTrainingJob`类创建AutoML训练管道，其中包括以下参数：

- `display_name`：`TrainingJob`资源的人可读名称。
- `prediction_type`：用于训练模型的任务类型。
  - `classification`：视频分类模型。
  - `object_tracking`：视频目标跟踪模型。
  - `action_recognition`：视频动作识别模型。

In [ ]:
job = aiplatform.AutoMLVideoTrainingJob(
    display_name="traffic",
    prediction_type="object_tracking",
)

print(job)

运行训练管道

接下来，您可以通过调用方法`run`来启动训练工作，使用以下参数：

- `dataset`: 用于训练模型的`Dataset`资源。
- `model_display_name`: 训练模型的可读名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试（留出数据）的数据集百分比。

当`run`方法完成时，会返回`Model`资源。

训练管道的执行将持续最多4小时。

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name="traffic",
    training_fraction_split=0.8,
    test_fraction_split=0.2,
)

## 检查模型评估分数
当您的模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。与数据集一样，您可以使用在部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
model_evaluations = model.list_model_evaluations()
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

In [ ]:
# Print the evaluation metrics
for evaluation in model_evaluations:
    evaluation = evaluation.to_dict()
    print("Model's evaluation metrics from Training:\n")
    metrics = evaluation["metrics"]
    for metric in metrics.keys():
        print(f"metric: {metric}, value: {metrics[metric]}\n")

发送一个批量预测请求，发送一个批量预测给部署的模型。

获取测试项目

现在对您的Vertex模型进行批量预测。您将使用数据集中的任意示例作为测试项目。不用担心这些示例可能已经用于训练模型 -- 我们只是想演示如何进行预测。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2
cols_1 = test_items[0].split(",")
cols_2 = test_items[1].split(",")
if len(cols_1) > 12:
    test_item_1 = str(cols_1[1])
    test_item_2 = str(cols_2[1])
    test_label_1 = str(cols_1[2])
    test_label_2 = str(cols_2[2])
else:
    test_item_1 = str(cols_1[0])
    test_item_2 = str(cols_2[0])
    test_label_1 = str(cols_1[1])
    test_label_2 = str(cols_2[1])

### 创建一个批处理输入文件

现在创建一个批处理输入文件，将其存储在您的本地云存储桶中。批处理输入文件可以是CSV格式或JSONL格式。在本教程中，您将使用JSONL格式。对于JSONL文件，您需要为每个视频的每一行创建一个字典条目。该字典包含以下键/值对：

- `content`：视频的云存储路径。
- `mimeType`：内容类型。在我们的示例中，是一个`avi`文件。
- `timeSegmentStart`：视频中要进行预测的起始时间戳。*注意*，时间戳必须指定为字符串，并后跟s（秒）、m（分钟）或h（小时）。
- `timeSegmentEnd`：视频中要进行预测的结束时间戳。

In [ ]:
test_filename = "test.jsonl"
gcs_input_uri = BUCKET_URI + "/test.jsonl"
# making data_1 and data_2 variables using the structure mentioned above
data_1 = {
    "content": test_item_1,
    "mimeType": "video/avi",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
}

data_2 = {
    "content": test_item_2,
    "mimeType": "video/avi",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
}

# getting reference to bucket
bucket = storage.Client(project=PROJECT_ID).bucket(BUCKET_URI.replace("gs://", ""))

# creating a blob
blob = bucket.blob(blob_name=test_filename)

# creating data variable
data = json.dumps(data_1) + "\n" + json.dumps(data_2) + "\n"

# uploading data variable content to bucket
blob.upload_from_string(data)

# printing path of uploaded file
print(gcs_input_uri)

# printing content of uploaded file
! gsutil cat $gcs_input_uri

### 发送批量预测请求

现在您的模型资源已经训练好，您可以通过调用batch_predict()方法来进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的易读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `sync`：如果设置为True，则调用将在等待异步批量作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="traffic",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

### 等待批量预测作业完成

接下来，请等待批量作业完成。另外，也可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

获取预测

接下来，从已完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的Cloud Storage输出存储桶中。您可以调用iter_outputs()方法来获取生成结果的每个Cloud Storage文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `id`：每个预测请求的内部分配的唯一标识符。
 - `displayName`：预测标签的类名称。
 - `confidences`：每个类别标签的预测置信度，介于0和1之间。
 - `timeSegmentStart`：视频中开始视频序列的时间偏移。
 - `timeSegmentEnd`：视频中结束视频序列的时间偏移。
 - `frames`：追踪对象的帧位置。

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}".replace(
        BUCKET_URI + "/", ""
    )
    data = bucket.get_blob(gfile_name).download_as_string()
    data = json.loads(data)
    print(data)

# 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

- 数据集
- 模型
- AutoML训练作业
- 批处理作业
- Cloud Storage存储桶

In [ ]:
# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI